In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
!pip install sacremoses

  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.6 MB/s eta 0:00:00a 0:00:01


In [3]:
from transformers import BioGptForSequenceClassification, BioGptTokenizer

# Load pre-trained model tokenizer and model
tokenizer = BioGptTokenizer.from_pretrained('microsoft/biogpt')
model = BioGptForSequenceClassification.from_pretrained('microsoft/biogpt', output_attentions=True)

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch

# Prepare data
def plots(text,filename):
    texts=[text]
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", return_attention_mask=True)

    with torch.no_grad():
        outputs = model(**inputs)

    attention = outputs.attentions[-1]
    attention_weights = attention.mean(dim=1)  
    diagonal_attention = attention_weights[:, range(attention_weights.size(1)), range(attention_weights.size(1))]
    # Example of averaging token attention weights for each sentence
    sentence_attention_weights = []
    for i, text in enumerate(texts):
        token_attention_weights = diagonal_attention[i].tolist()
        average_attention = sum(token_attention_weights) / len(token_attention_weights)
        sentence_attention_weights.append(average_attention)

    attention_data = []
    for i, text in enumerate(texts):
        tokens = tokenizer.tokenize(text)
        weights = diagonal_attention[i].tolist()[:len(tokens)]  # Truncate to the length of the number of tokens
        sentence_att_weight = sentence_attention_weights[i]  # Sentence-level attention weight
        for token, weight in zip(tokens, weights):
            attention_data.append({
                'sent_id': text,
                'raw_word': token,
                'normalized_word_weight': weight,
                'sent_att_weight': sentence_att_weight
            })

    attention_df = pd.DataFrame(attention_data)
    predict = 0.01
    context_series = pd.Series({'label': 'Positive', 'name': 'Patient History'})
    normalized_attention_df = attention_df.copy()
    normalized_attention_df['normalized_word_weight'] = normalized_attention_df['normalized_word_weight'] / normalized_attention_df['normalized_word_weight'].max()
    filtered_df = normalized_attention_df[normalized_attention_df['normalized_word_weight'] > predict]
    print(filtered_df.head())
    return filtered_df

In [5]:
patient_histories = pd.read_csv("/kaggle/input/complete-dataset/Patient_Histories_Cleaned.csv")
attention_patient_history=[]
for index, row in patient_histories.iterrows():
        results = []
        patient_id = index+1
        patient_history = row['Patient History']
        diagnosis = row['Diagnosis']
        attention_patient_history.append(plots(patient_history,f'patient-{patient_id}-BioBert-Attention.png'))

                                             sent_id  raw_word  \
0  A 30-year-old female presents with bloody disc...     A</w>   
1  A 30-year-old female presents with bloody disc...    30</w>   
2  A 30-year-old female presents with bloody disc...   @-@</w>   
3  A 30-year-old female presents with bloody disc...  year</w>   
4  A 30-year-old female presents with bloody disc...   @-@</w>   

   normalized_word_weight  sent_att_weight  
0                1.000000         0.052298  
1                0.102997         0.052298  
2                0.110577         0.052298  
3                0.038693         0.052298  
4                0.048581         0.052298  
                                             sent_id  raw_word  \
0  A 65-year-old woman arrives to the ED complain...     A</w>   
1  A 65-year-old woman arrives to the ED complain...    65</w>   
2  A 65-year-old woman arrives to the ED complain...   @-@</w>   
3  A 65-year-old woman arrives to the ED complain...  year</w>   
4  

In [6]:
patient_history_list = []
for df in attention_patient_history:
    df['raw_word']= df['raw_word'].str.replace('</w>', '')
    patient_history_list.append(' '.join(df['raw_word'].astype(str).tolist()))

patient_history_df = pd.DataFrame(patient_history_list, columns=['CombinedRawWords'])


In [7]:
!pip install sacremoses

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

In [8]:
summarized_abstracts_path = '/kaggle/input/complete-dataset/summarized_abstracts_gpt'

In [9]:
from transformers import AutoTokenizer
bootstrapped_model_name = "stanford-crfm/BioMedLM"
tokenizer = AutoTokenizer.from_pretrained(bootstrapped_model_name)

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=bootstrapped_model_name, tokenizer=tokenizer)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
    
    return predicted_class_idx

MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

def create_input_text(question, context):
    input_text =  "Is this patient history: "+ context + "related to abstract: " + question
    return input_text

def create_df(patient_id,indexes, pubids, results):
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    return df
    
label_mapping = {
    "LABEL_0": "yes",
    "LABEL_1": "no",
    "LABEL_2": "maybe"
}

def map_label(result_str):
    try:
        result_str = result_str.replace("'", '"')
        result = json.loads(result_str)
        label = result[0]['label']
        return label_mapping.get(label, "Unknown")
    except:
        return "Error in processing"

In [11]:
def extract_patient_number(filename):
    parts = filename.split('-')
    for part in parts:
        if part.isdigit():
            return part
    return none

In [12]:
summarized_abstracts_path = '/kaggle/input/complete-dataset/summarized_abstracts_gpt'

In [13]:
# Loop to process and handle each patient's data individually
for file in os.listdir('/kaggle/input/complete-dataset/BootStrapped-Level-2-GPT'):
    if file.endswith('.csv'):
        patient_number = extract_patient_number(file)
        if patient_number is none:
            continue

        # Load and filter the result file
        result_df = pd.read_csv(os.path.join('/kaggle/input/complete-dataset/BootStrapped-Level-2-GPT', file))
        filtered_results = result_df[result_df['Result'].isin(['yes', 'maybe'])]

        # Load and filter the abstracts file for this patient
        abstracts_filename = f"summarized_patient-number{patient_number}-articles.csv"
        abstracts_path = os.path.join(summarized_abstracts_path, abstracts_filename)
        if os.path.exists(abstracts_path):
            abstracts_df = pd.read_csv(abstracts_path)
            filtered_abstracts = abstracts_df[abstracts_df['ID'].isin(filtered_results['PubID'])]
        else:
            filtered_abstracts = pd.DataFrame()
        # Get the diagnosis for this patient from the patient history file
        history = patient_history_list[int(patient_number)-1]
#         patient_history= 
#         print(patient_history)
        # Combine the results and abstracts into one DataFrame and add the diagnosis as a new column
        combined_df = pd.merge(filtered_results, filtered_abstracts, left_on='PubID', right_on='ID', how='left')
        combined_df['History'] = history
        combined_df['input_text'] = combined_df.apply(lambda row: create_input_text(row['Summary'], row['History']), axis=1)
        results = []
        pubids = []
        indexes = []
        for idx, row in combined_df.iterrows():
            indexes.append(idx + 1)
            pubids.append(row['PubID'])
            results.append(classifier(truncate_text_to_fit(row['input_text'])))

        if results:
            result=create_df(patient_number,indexes, pubids, results)
            result['Result'] = result['Result'].apply(map_label)
            new_file_name = f"/kaggle/working/Patient-{patient_number}-final-results-BioGpt.csv"
            combined_df.to_csv(new_file_name, index=False)
            print(f"Processed, and saved: {new_file_name}")

Processed, and saved: /kaggle/working/Patient-6-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-13-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-5-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-10-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-4-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-7-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-1-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-16-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-8-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-12-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-15-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-9-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient-14-final-results-BioGpt.csv
Processed, and saved: /kaggle/working/Patient